In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26771486
paper_name = 'ast_schuldiner_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [16]:
original_data = pd.read_excel('raw_data/mmc2.xlsx', sheet_name='Table S1', skiprows=1)

In [17]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5234 x 5


In [18]:
original_data.head()

,ORF,Gene,Growth with control (mCherry) (AU),Growth with clogger (AU),Statistically Significant Differential Hits
0,YGR105W,VMA21,0.354617,0.076074,NaN
1,YBR020W,GAL1,0.240696,0.080709,NaN
2,YOR302W,0,0.106188,0.087154,NaN
3,YOR302W,0,1.290085,0.101217,Statistically significant worse growth
4,YBR018C,GAL7,0.059551,0.103550,NaN


In [19]:
original_data['orf'] = original_data['ORF'].astype(str)

In [20]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [22]:
typo_fixes = {'YOLO57W':'YOL057W','YBRF182C-A':'YBR182C-A', 'YJL206-A': 'YJL206C-A',
              'YKLO72W':'YKL072W','YOLO62C':'YOL062C','YLR287-A':'YLR287C-A'}

In [23]:
for typo in typo_fixes.keys():
    original_data.loc[original_data['orf']==typo, 'orf'] = typo_fixes[typo]

In [24]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, Growth with control (mCherry) (AU), Growth with clogger (AU), Statistically Significant Differential Hits, orf]
Index: []


In [26]:
original_data['data'] = original_data['Growth with clogger (AU)'] / original_data['Growth with control (mCherry) (AU)']

In [27]:
original_data.set_index('orf', inplace=True)

In [28]:
original_data = original_data[['data']].copy()

In [29]:
original_data = original_data.groupby(original_data.index).mean()

In [30]:
original_data.shape

(5085, 1)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [32]:
dataset_ids = [11821]
datasets = datasets.reindex(index=dataset_ids)

In [33]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [34]:
data.head()

dataset_id,11821
data_type,value
orf,
YAL002W,1.105972
YAL003W,0.467464
YAL004W,1.443783
YAL005C,1.639496
YAL007C,0.842791


## Subset to the genes currently in SGD

In [35]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [36]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [37]:
data.head()

,dataset_id,11821
,data_type,value
gene_id,orf,
2,YAL002W,1.105972
3,YAL003W,0.467464
1863,YAL004W,1.443783
4,YAL005C,1.639496
5,YAL007C,0.842791


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [41]:
data_all.head()


dataset_id          11821          
data_type           value    valuez
gene_id orf                        
2       YAL002W  1.105972  0.463405
3       YAL003W  0.467464 -2.087962
1863    YAL004W  1.443783  1.813240
4       YAL005C  1.639496  2.595275
5       YAL007C  0.842791 -0.588221

# Print out

In [42]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [43]:
from IO.save_data_to_db3 import *

In [44]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 26771486...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.13s/it]

Updating the data_modified_on field...


In [45]:
data_all.shape

(5059, 2)